In [ ]:
# cross_val_predict, StratifiedKFold, roc_auc_score, KNeighborsClassifier, LogistricRegression, SVC
# XGBClassifier, titanic

# Stacking

Несмотря на то, что в открытом доступе существует довольно много реализаций стекинга, некоторые из которых даже представлены в виде библиотечных функций, лучше сделать собественную. Стекинг - не классический алгоритм решения задачи, а скорее набор правил для помощи в решении задачи другим алгоритмам. Если вы серьезно займетесь машинным обучением, рано или поздно вам скорее всего захочется что-нибудь поменять в этом наборе правил, поэтому собственная реализация с понятным вам кодом будет как нельзя кстати.

В этом домашнем задании вы создадите собственную функцию для блендинга/классического стекинга моделей, которая пригодится вам в дальнейшем.

Ниже вы увидите заготовку для функции, где:

- models - список объектов базовых алгоритмов

- meta_alg - мета-алгоритм

data_train, targets_train, data_test, targets_test - тренировочные и тестовые признаки и целевые переменные

- test_size - размер тестовых данных для блендинга в промежутке (0, 1)

- cv - количество разбиений для кросс-валидации

In [1]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
#def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, test_size=None, cv=5):

### 1.
В следующей ячейке в теле функции определен условный оператор if-else. После else вместо pass пропишите код из лекции с некоторыми новыми вставками в таком порядке: деление data_train и targets_train на тренировочные и валидационные данные с помощью функции train_test_split, где test_size=test_size из определения функции; определение матрицы meta_mtrx; цикл, в котором заполняется meta_mtrx: сначала строка, где модель обучается с помощью метода fit на тренировочных данных из разбиения, затем строка, где матрица заполняется значениями предсказаний моделей на валидационных данных. На этом цикл заканчивается.

После цикла добавьте строку с методом fit мета-алгоритма: обучите его на значениях полученной матрицы meta_mtrx и целевой переменной для валидационных данных.

Определите матрицу meta_mtrx_test. Напишите цикл, где эта матрица заполняется предсказаниями базовых моделей на тестовых данных data_test.

После цикла сделайте предсказания мета-алгоритма для значений матрицы meta_mtrx_test.

Дополните код еще одним оператором if, который будет проверять, существуют ли данные targets_test для проверки качества работы модели на тестовых данных: если targets_test не является None, тогда выведите метрику roc_auc_score для предсказаний мета-алгоритма на тестовых данных.

In [3]:
def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, random_state=None, test_size=None, cv=5):
    if test_size is None:
        meta_mtrx = np.empty((data_train.shape[0], len(models)))
        
        for n, model in enumerate(models):
            meta_mtrx[:, n] = cross_val_predict(model, data_train, targets_train, cv=cv, method='predict')
            model.fit(data_train, targets_train)
            print(meta_mtrx[:,n])
            
        meta_alg.fit(meta_mtrx, targets_train)
            
        meta_mtrx_test = np.empty((x_test.shape[0],len(models)))
        
        for n, model in enumerate(models):
            meta_mtrx_test[:, n] = model.predict(data_test)
            
        meta_predict = meta_alg.predict(meta_mtrx_test)
            
        if targets_test is not None:
            print(f'Стекинг roc_auc = {roc_auc_score(targets_test, meta_predict)}')
            
    elif test_size > 0 and test_size < 1:
        x_train_local, x_test_local, y_train_local, y_test_local = train_test_split(data_train, targets_train, test_size=test_size, random_state=1)
        meta_mtrx = np.empty((x_test_local.shape[0], len(models)))
        
        for n, model in enumerate(models):
            model.fit(x_train_local, y_train_local)
            meta_mtrx[:, n] = model.predict(x_test_local)
        
        meta_alg.fit(meta_mtrx, y_test_local)
        
        meta_mtrx_test = np.empty((x_test.shape[0], len(models)))
        
        for n, model in enumerate(models):
            meta_mtrx_test[:, n] = model.predict(data_test)
        
        meta_predict = meta_alg.predict(meta_mtrx_test)
        
        if targets_test is not None:
            roc_auc_score(targets_test, meta_predict)
        print(f'Стекинг roc_auc = {roc_auc_score(targets_test, meta_predict)}')
        
    else:
        raise ValueError("test_size must be between 0 and 1")        

### 2.
Теперь напишите код стекинга вместо pass после оператора if.

Сразу определите матрицу meta_mtrx. Напишите цикл для заполнения матрицы: сначала напишите строку, где каждый столбец метапризнаков (для каждой модели) заполняется с помощью функции cross_val_predict(base_algotithm, data_train, targets_train, cv, method='predict'); следующая строка - обучите каждый базовый алгоритм на полном тренировочном датасете.

После цикла обучите мета-алгоритм на матрице метапризнаков meta_mtrx. Определите матрицу meta_mtrx_test.

Напишите второй цикл, где матрица meta_mtrx_test заполняется предсказаниями базовых моделей на тестовых данных data_test.

После цикла сделайте предсказания мета-алгоритма для значений матрицы meta_mtrx_test.

Так же, как и для блендинга, напишите код проверки для targets_test и выведите roc_auc_score, если это возможно.

In [4]:
# ваш код

### 3.
Базовая функция стекинга готова. Теперь проверим, как она работает. Ниже представлен уже знакомый нам датасет Titanic, разделенный на тренировочный и тестовый датасеты; предопределенные базовые алгоритмы и мета-алгоритм. Ваша задача - составить список базовых алгоритмов и запустить функцию в трех различных вариантах (при этом в каждом из них все значения data_train, targets_train, data_test, targets_test должны быть определены):

1. Вызвать исключение "test_size must be between 0 and 1".

2. Установить test_size=0.3; вывести AUC и массив полученных предсказаний.

3. Оставить test_size=None; вывести AUC и массив полученных предсказаний.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier


titanic = pd.read_csv('./data/titanic_uQHiUzi.csv')
targets = titanic.Survived
data = titanic.drop(columns='Survived')

x_train, x_test, y_train, y_test = train_test_split(data, 
                                                    targets,
                                                    train_size=0.8,
                                                    random_state=17)

knn = KNeighborsClassifier(n_neighbors=3)
lr = LogisticRegression(random_state=17)
svc = SVC(random_state=17)

meta = XGBClassifier(n_estimators=40)

In [5]:
x_train.shape

(712, 7)

In [8]:
# ваш код

# п.1
models = [knn, lr, svc]
stacking(models, meta, x_train, y_train, x_test, y_test, test_size=99, random_state=1)

ValueError: test_size must be between 0 and 1

In [9]:
# п.2
stacking(models, meta, x_train, y_train, x_test, y_test, test_size=0.3, random_state=1)

Стекинг roc_auc = 0.7580124063065392


In [9]:
# п.3
stacking(models, meta, x_train, y_train, x_test, y_test, random_state=1)

(712, 3)
Стекинг roc_auc = 0.7601447402429569


По мере того, как вы будете использовать эту функцию, вам могут пригодиться такие дополнительные параметры как: random_state, который позволит делать воспроизводимые модели; metrics - список метрик, которые могут быть вычислены; grid_search, который может производить поиск лучших параметров для алгоритмов, и т.д.